# Capstone Project The Battle of Neighborhoods (Week 2)

## Problem Description
Nowadays, there are a lot of food bloggers in each and every city, who go around discovering places to fillup our stomach and heart everytime we feel hungry.<br>
So here I planned to group together places where you would find different genres of food. <br>

## Data Description
The dataset to be used would be the New-York dataset which was previously used in Optional Assignment of Week-2.<br>
The types of establishments I'll be targeting are (but not limited to) restaurants, cafes, bars, delis, bakeries, bistros and more.<br>
I plan segregate these places from others for each borough, and group them together. If possible, I may also provide a ranking for each cluster.

Loading the libarries needed for the project, alongwith my Foursquare API credentials

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


CLIENT_ID = 'GOB3JCHBYQXPYCANCHMHZFU1JD3JHVQ1JKTDUYKSZQRGG1IH' # your Foursquare ID
CLIENT_SECRET = 'TYXODU3E0FYDVNFBAECEEV2AMDBIK0NYPYRN5MH2FBKNFEVZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print("Credentials Approved !")

Libraries imported.
Credentials Approved !


Loading the JSON file

In [2]:
#!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


Creating a dataframe for the data

In [3]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods_data = newyork_data['features']

Loading data into the dataframe

In [5]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Shuffling the rows within the dataframe<br>(This step is totally optional)<br>Then we view how the dataframe looks like

In [6]:
neighborhoods = neighborhoods.sample(frac=1).reset_index(drop=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Carnegie Hill,40.782683,-73.953256
1,Staten Island,Sandy Ground,40.541140,-74.217766
2,Brooklyn,Ocean Hill,40.678403,-73.913068
3,Queens,Hillcrest,40.723825,-73.797603
4,Queens,Little Neck,40.770826,-73.738898


In [7]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 1500 # define radius

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
print("There are {} many boroughs namely\n {}\nChoose one :".format(len(neighborhoods.Borough.unique()),neighborhoods.Borough.unique()))
u_choice = input("Enter name of Borough > ")
if u_choice not in neighborhoods.Borough.unique():
    while( u_choice not in neighborhoods.Borough.unique()):
        print("Please enter the exact name :")
        u_choice = input("Enter name of Borough > ")

There are 5 many boroughs namely
 ['Manhattan' 'Staten Island' 'Brooklyn' 'Queens' 'Bronx']
Choose one :
Enter name of Borough > Brooklyn


In [14]:
choice = neighborhoods['Borough'].values == u_choice
choice = neighborhoods[choice]
ny_venues = getNearbyVenues(names=choice.Neighborhood,latitudes=neighborhoods.Latitude,longitudes=neighborhoods.Longitude,radius=radius)

Ocean Hill
Gowanus
Fort Greene
Marine Park
Dyker Heights
Remsen Village
Bergen Beach
Windsor Terrace
Coney Island
Ocean Parkway
Boerum Hill
Paerdegat Basin
Highland Park
Prospect Park South
East Flatbush
Bedford Stuyvesant
Cypress Hills
Mill Basin
Park Slope
Gravesend
Fulton Ferry
Mill Island
Carroll Gardens
East Williamsburg
New Lots
Bath Beach
South Side
Prospect Lefferts Gardens
Rugby
Starrett City
Borough Park
Cobble Hill
North Side
Brownsville
Bushwick
Bay Ridge
Williamsburg
Broadway Junction
Homecrest
Bensonhurst
Gerritsen Beach
East New York
Sea Gate
Greenpoint
City Line
Madison
Midwood
Wingate
Fort Hamilton
Ditmas Park
Red Hook
Manhattan Beach
Manhattan Terrace
Sheepshead Bay
Dumbo
Georgetown
Erasmus
Crown Heights
Canarsie
Downtown
Brooklyn Heights
Kensington
Flatbush
Vinegar Hill
Sunset Park
Clinton Hill
Brighton Beach
Prospect Heights
Weeksville
Flatlands


In [15]:
f_type = ['Restaurant','Café','Diner','Bakery',
          'Coffee','Pizza','Donut','Sandwich','Bar','Pub','Tea']

In [16]:
#venue_list=[]
city_list=[]
for item in f_type:
    city_list.append([s for s in ny_venues['Venue Category'].unique() if item.lower() in s.lower()])
#city_list
print("We've few options for your food cravings ! What do you want ?")
print(f_type)
u_choice = input('Enter your choice :  ').lower()

We've few options for your food cravings ! What do you want ?
['Restaurant', 'Café', 'Diner', 'Bakery', 'Coffee', 'Pizza', 'Donut', 'Sandwich', 'Bar', 'Pub', 'Tea']
Enter your choice :  Coffee


In [17]:
venue_list = [s for s in ny_venues['Venue Category'].unique() if u_choice.lower() in s.lower()]
venues = ny_venues.loc[ny_venues['Venue Category'] == venue_list[0]]

In [18]:
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('New York')
latitude = location.latitude
longitude = location.longitude
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood,v_name  in zip(venues['Venue Latitude'],venues['Venue Longitude'],venues['Neighborhood'], venues['Venue']):
    label = '{}, {}'.format(v_name, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork